In-sample

In [ ]:
!pip install transformers

In [ ]:
# from transformers import RobertaTokenizer, RobertaModel
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import pandas as pd
from torch.nn.functional import softmax
import torch
import numpy as np
import nltk
import string
from transformers import pipeline

In [ ]:
final_news_df = pd.read_csv('/content/drive/MyDrive/outsamp_headlines.csv')
final_news_df.clean_headline = final_news_df.clean_headline.fillna('')

In [ ]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from transformers import pipeline

# Load the BERT sentiment analysis model
model_name = "bert-large-uncased"
sentiment_analyzer = pipeline("sentiment-analysis", model=model_name)
# X = final_news_df['clean_headline'].to_list()
final_news_df['clean_headline'] = [x[:512] for x in final_news_df['clean_headline']]
# y = final_news_df['sentiment'].to_list()
labels = []
scores = []
lyst = final_news_df.clean_headline.tolist()
result = sentiment_analyzer(lyst)
# scores = result[0]["score"]
# Perform sentiment analysis on each preprocessed headline and store the results in a new column
for i in result:
  scores.append(result[0]['score'])
  labels.append(result[0]['label'])


final_news_df["Sentiment_Score"] = scores
final_news_df["Sentiment"] = labels
final_news_df["Sentiment"] = final_news_df["Sentiment"].apply(lambda x: 'positvie' if x == "LABEL_1" else 'negative')


# Print the DataFrame with sentiment analysis results
# print(final_news_df)

# Calculate the true binary labels for sentiment
# Assuming you have a column named "True_Sentiment" with labels "Positive," "Negative," "Neutral"
# true_labels = final_news_df["True_Sentiment"].apply(lambda x: 1 if x == "Positive" else 0)
final_news_df.to_csv('/content/drive/MyDrive/outsamp_fb_head_output.csv', index=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sentiment_analyzer("I love the rain and sunshine, and the weather is good, I feel so happy")[0]['label']

'LABEL_0'

In [ ]:
final_news_df.to_csv('/content/drive/MyDrive/outsamp_b_head_output.csv', index=False)

In [ ]:
X = final_news_df['clean_headline'].to_list()
X = [x[:512] for x in X]
y = final_news_df['sentiment'].to_list()

print(X)
    # print(x, '----', val)
    # print('#######################################################')
    # sent_val.append(val)

# print(accuracy_score(y, sent_val))

['kinem enter strateg proteom biomark discoveri collabor kinem enter strateg proteom biomark discoveri collabor kgaa darmstadt germani syndax pharmaceut announc collabor evalu combin avelumab entinostat ovarian cancer kgaa darmstadt germani syndax announc collabor evalu combin avelumab entinostat ovarian cancer sach associ ltd annual european life scienc ceo forum exhibit mar mar present annual european life scienc ceo forum exhibit mar present jpmorgan annual healthcar confer jan biocom sa biocom global life', 'corpor present jpmorgan annual healthcar confer jan life scienc nation redefin earli stage invest resi confer jan corpor present redefin earli stage invest resi confer jan well fargo secur llc well fargo secur healthcar confer sep sep ub invest bank ub global healthcar confer may may corpor announc preliminari sale result fourth quarter full year provid earn guidanc full year corpor announc preliminari sale result fourth quarter full year provid earn guidanc full year tranch up

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

labels = {0:'neutral', 1:'positive',2:'negative'}

sent_val = list()
sentiment_scores_out = []
for x in range(len(X)):
    inputs = tokenizer(x, return_tensors="pt", padding=True, truncation=True, max_length=555)
    outputs = finbert(**inputs)
    logits = outputs.logits
    probs = softmax(logits, dim=1)
    val = probs.detach().numpy()
    sentiment_scores_out.append(val)

final_news_df["positive"] = sentiment_scores_out[:, 1]  # Use index 1 for positive sentiment
final_news_df["negative"] = sentiment_scores_out[:, 0]

In [ ]:
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', num_labels=3)
# model = RobertaModel.from_pretrained('roberta-base')
model_name = "ProsusAI/finbert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
def predict_sentiment_scores(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    probs = softmax(logits, dim=1)
    return probs.detach().numpy()

In [ ]:
headlines = final_news_df["clean_headline"].tolist()
sentiment_scores = predict_sentiment_scores(headlines)

In [ ]:
# Append sentiment scores to the DataFrame
final_news_df["positive"] = sentiment_scores[:, 1]  # Use index 1 for positive sentiment
final_news_df["negative"] = sentiment_scores[:, 0]
final_news_df.to_csv('/content/drive/MyDrive/insamp_fb_head_output.csv', index=False)

Out-Sample

In [ ]:
final_news_df_out = pd.read_csv('/content/drive/MyDrive/outsamp_headlines.csv')
final_news_df_out.clean_headline = final_news_df_out.clean_headline.fillna('')

In [ ]:
headlines_out = final_news_df_out["clean_headline"].tolist()
sentiment_scores_out = predict_sentiment_scores(headlines_out)

# Append sentiment scores to the DataFrame
final_news_df_out["positive"] = sentiment_scores_out[:, 1]  # Use index 1 for positive sentiment
final_news_df_out["negative"] = sentiment_scores_out[:, 0]
final_news_df_out.to_csv('/content/drive/MyDrive/outsamp_fb_head_output.csv', index=False)